# SMI AL Loop

In [2]:
import h5py
import time
import random
import datetime
import copy
import numpy as np
import os
import subprocess
import sys
import json
import PIL.Image as Image
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from matplotlib import pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from cords.cords.selectionstrategies.supervisedlearning import DataSelectionStrategy
from cords.cords.utils.models import ResNet18
from gable.gable.utils.custom_dataset import load_dataset_custom
from torch.utils.data import Subset
from torch.autograd import Variable
import tqdm
from math import floor
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
seed=42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
# for cuda
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.enabled = False

In [3]:
from torch.utils.data import Dataset
class custom_subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
        labels(sequence) : targets as required for the indices. will be the same length as indices
    """
    def __init__(self, dataset, indices, labels):
        self.dataset = torch.utils.data.Subset(dataset, indices)
        self.targets = labels.type(torch.long)
    def __getitem__(self, idx):
        image = self.dataset[idx][0]
        target = self.targets[idx]
        return (image, target)

    def __len__(self):
        return len(self.targets)

In [4]:
def model_eval_loss(data_loader, model, criterion):
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss

def init_weights(m):
#     torch.manual_seed(35)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()
                
def create_model(name, num_cls, device):
    if name == 'ResNet18':
        model = ResNet18(num_cls)
    elif name == 'MnistNet':
        model = MnistNet()
    elif name == 'ResNet164':
        model = ResNet164(num_cls)
    model.apply(init_weights)
    model = model.to(device)
    return model

def loss_function():
    criterion = nn.CrossEntropyLoss()
    criterion_nored = nn.CrossEntropyLoss(reduction='none')
    return criterion, criterion_nored

def optimizer_with_scheduler(model, num_epochs, learning_rate, m=0.9, wd=5e-4):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    return optimizer, scheduler

def optimizer_without_scheduler(model, learning_rate, m=0.9, wd=5e-4):
#     optimizer = optim.Adam(model.parameters(),weight_decay=wd)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    return optimizer

def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing/3600

def kernel(x, y, measure="cosine", exp=2):
    if(measure=="eu_sim"):
        lam = 0.25
        dist = pairwise_distances(x.cpu().numpy(), y.cpu().numpy())
        sim = max(dist.ravel()) - dist
        sim *= lam
        sim = np.exp(sim)
#         n = x.size(0)
#         m = y.size(0)
#         d = x.size(1)
#         x = x.unsqueeze(1).expand(n, m, d)
#         y = y.unsqueeze(0).expand(n, m, d)
#         dist = torch.pow(x - y, exp).sum(2)
#         const = torch.max(dist).item()
#         sim = (const - dist)
    
        #dist = torch.exp(-1 * torch.pow(x - y, 2).sum(2))
    if(measure=="cosine"):
        sim = cosine_similarity(x.cpu().numpy(), y.cpu().numpy())
        sim = np.exp(sim)
    return sim


def save_kernel_hdf5(lake_kernel, lake_target_kernel, target_kernel=[], numpy=True):
    if(not(numpy)):
        lake_kernel = lake_kernel.cpu().numpy()
    with h5py.File("lake_kernel.hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=lake_kernel)
    if(not(numpy)):
        lake_target_kernel = lake_target_kernel.cpu().numpy()
    with h5py.File("lake_target_kernel.hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=lake_target_kernel)
    if(not(numpy)):
        target_kernel = target_kernel.cpu().numpy()
    with h5py.File("target_kernel.hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=target_kernel)
            
def find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, 
                       final_tst_predictions, saveDir, prefix):
    #find queries from the validation set that are erroneous
    saveDir = os.path.join(saveDir, prefix)
    if(not(os.path.exists(saveDir))):
        os.mkdir(saveDir)
    val_err_idx = list(np.where(np.array(final_val_classifications) == False)[0])
    tst_err_idx = list(np.where(np.array(final_tst_classifications) == False)[0])
    class_err_log = []
    val_class_err_idxs = []
    tst_err_log = []
    for i in range(num_cls):
        tst_class_idxs = list(torch.where(torch.Tensor(test_set.targets) == i)[0].cpu().numpy())
        val_class_idxs = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        #err classifications per class
        val_err_class_idx = set(val_err_idx).intersection(set(val_class_idxs))
        tst_err_class_idx = set(tst_err_idx).intersection(set(tst_class_idxs))
        print("val, test error% for class ", i, " : ", round((len(val_err_class_idx)/len(val_class_idxs))*100,2), round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2))
        val_class_err_idxs.append(val_err_class_idx)
        class_err_log.append("val, test error% for class "+ str(i) + " : "+ str(round((len(val_err_class_idx)/len(val_class_idxs))*100,2)) + ", " + str(round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2)))
        tst_err_log.append(round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2))
        
    return tst_err_log, val_class_err_idxs


def aug_train_subset(train_set, lake_set, subset, lake_subset_idxs, budget):
    all_lake_idx = list(range(len(lake_set)))
    if(not(len(subset)==budget)):
        print("Budget not filled, adding ", str(int(budget) - len(subset)), " randomly.")
        remain_budget = int(budget) - len(subset)
        remain_lake_idx = list(set(all_lake_idx) - set(subset))
        random_subset_idx = list(np.random.choice(np.array(remain_lake_idx), size=int(remain_budget), replace=False))
        subset += random_subset_idx
    lake_ss = custom_subset(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
    remain_lake_idx = list(set(all_lake_idx) - set(lake_subset_idxs))
    remain_lake_set = custom_subset(lake_set, remain_lake_idx, torch.Tensor(lake_set.targets.float())[remain_lake_idx])
    assert((len(lake_ss)+len(remain_lake_set))==len(lake_set))
    aug_train_set = torch.utils.data.ConcatDataset([train_set, lake_ss])
    return aug_train_set, remain_lake_set, subset
                        
def getMisclsSet(val_set, val_class_err_idxs, imb_cls_idx):
    miscls_idx = []
    for i in range(len(val_class_err_idxs)):
        if i in imb_cls_idx:
            miscls_idx += val_class_err_idxs[i]
    print("total misclassified ex from imb classes: ", len(miscls_idx))
    return Subset(val_set, miscls_idx)

def getPrivateSet(lake_set, subset, private_set):
    #augment prev private set and current subset
    new_private_set = custom_subset(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
#     new_private_set =  Subset(lake_set, subset)
    total_private_set = torch.utils.data.ConcatDataset([private_set, new_private_set])
    return total_private_set

def getSMI_ss(datkbuildPath, exePath, hdf5Path, budget, numQueries, sf):
    if(sf=="fl1mi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "lake_kernel.hdf5") +  " -queryKernelFile " + os.path.join(hdf5Path, "lake_target_kernel.hdf5")
    elif(sf == "logdetmi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -logDetLambda 1 -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries  " + numQueries + "  -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "lake_kernel.hdf5") + " -queryKernelFile " + os.path.join(hdf5Path, "lake_target_kernel.hdf5") + " -queryqueryKernelFile " + os.path.join(hdf5Path, "target_kernel.hdf5")
    elif(sf=="fl2mi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -queryDiversityLambda 1 -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries  " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "lake_kernel.hdf5") + " -queryKernelFile " + os.path.join(hdf5Path, "lake_target_kernel.hdf5")
    elif(sf=="gcmi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5") + " -queryKernelFile " + os.path.join(hdf5Path,"lake_target_kernel.hdf5")
    elif(sf=="gccg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -gcLambda 1 -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"lake_target_kernel.hdf5")
    elif(sf=="fl1cg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"lake_target_kernel.hdf5")
    elif(sf=="logdetcg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -magnificationLambda " + str(magnification) + " -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"lake_target_kernel.hdf5") + " -privateprivateKernelFile " + os.path.join(hdf5Path, "target_kernel.hdf5")
    elif(sf=="fl" or sf=="logdet"):
        command = os.path.join(datkbuildPath, "cifarSubsetSelector_ng") + " -mode generic -naiveOrRandom naive -logDetLambda 1 -numSummaries 1 -budget " + str(budget) + " -genericOptimizer " + sf + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5")
    elif(sf =="gc"):
        command = os.path.join(datkbuildPath, exePath) + " -mode generic -naiveOrRandom naive -gcLambda 1 -numSummaries 1 -budget " + str(budget) + " -genericOptimizer " + sf + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"lake_kernel.hdf5")
    print("Executing SIM command: ", command)
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=True, shell=True)
    subset = process.communicate()[0]
    subset = subset.decode("utf-8")
    subset = subset.strip().split(" ")
    subset = list(map(int, subset))
    return subset

# def getDuplicates(subset):
    
#check overlap with prev selections
def check_overlap(prev_idx, prev_idx_hist, idx):
    prev_idx = [int(x/num_rep) if x < ((split_cfg["num_rep"] * split_cfg["lake_subset_repeat_size"])-1) else x for x in prev_idx ]
    prev_idx_hist = [int(x/num_rep) if x < ((split_cfg["num_rep"] * split_cfg["lake_subset_repeat_size"])-1) else x for x in prev_idx_hist]
    idx = [int(x/num_rep) if x < ((split_cfg["num_rep"] * split_cfg["lake_subset_repeat_size"])-1) else x for x in idx]
    # overlap = set(prev_idx).intersection(set(idx))
    overlap = [value for value in idx if value in prev_idx] 
    # overlap_hist = set(prev_idx_hist).intersection(set(idx))
    overlap_hist = [value for value in idx if value in prev_idx_hist]
    new_points = set(idx) - set(prev_idx_hist)
    total_unique_points = set(idx+prev_idx_hist)
    print("Num unique points within this selection: ", len(set(idx)))
    print("New unique points: ", len(new_points))
    print("Total unique points: ", len(total_unique_points))
    print("overlap % of sel with prev idx: ", len(overlap)/len(idx))
    print("overlap % of sel with all prev idx: ", len(overlap_hist)/len(idx))
#     return len(overlap)/len(idx), len(overlap_hist)/len(idx)
    return len(total_unique_points)


In [5]:
datadir = 'data/'
data_name = 'cifar100'
num_cls=10
fraction = float(0.1)
budget=500
num_epochs = int(10)
num_rep = 10
# feature='vanilla'
feature = 'duplicate'
# feature = 'classimb'
num_runs = 1  # number of random runs
learning_rate = 0.01
model_name = 'ResNet18'
magnification = 10
device = "cuda:1" if torch.cuda.is_available() else "cpu"
# split_cfg = {"num_cls_imbalance":2, "per_imbclass_train":50, "per_imbclass_val":25, "per_imbclass_lake":150, "per_class_train":1000, "per_class_val":25, "per_class_lake":3000}
split_cfg = {"train_size":500, "val_size":1000, "lake_size":5000, "num_rep":num_rep, "lake_subset_repeat_size":1000}
datkbuildPath = "/home/snk170001/bioml/dss/notebooks/datk/build"
exePath = "cifarSubsetSelector"
initModelPath = "weights/cg_" + data_name + "_" + model_name + "_" + str(learning_rate) + "_" + str(split_cfg["train_size"])
print("Using Device:", device)
doublePrecision = False
linearLayer = False

Using Device: cuda:1


In [6]:
from distil.distil.active_learning_strategies import BADGE, EntropySampling, GLISTER
from distil.distil.utils.DataHandler import DataHandler_CIFAR10

# AL Like Train Loop

In [7]:
def train_model_al(datkbuildPath, exePath, num_epochs, dataset_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run,
                device, computeErrorLog, strategy="SIM", sf=""):
#     torch.manual_seed(42)
#     np.random.seed(42)
    print(strategy, sf)
    #load the dataset based on type of feature
    if(feature=="classimb"):
        train_set, val_set, test_set, lake_set, imb_cls_idx, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg)
        print("imbalanced classes are: ", imb_cls_idx)
    if(feature=="duplicate" or feature=="vanilla"):
        sel_cls_idx = None
        if(strategy == "SIM" or strategy=="random"):
            train_set, val_set, test_set, lake_set, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg)
        elif(strategy=="AL"):
            X_tr, y_tr, X_val, y_val, X_unlabeled, y_unlabeled, train_set, val_set, test_set, lake_set, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, True, False)
        
    N = len(train_set)
    trn_batch_size = 20
    val_batch_size = 10
    tst_batch_size = 100

    trainloader = torch.utils.data.DataLoader(train_set, batch_size=trn_batch_size,
                                              shuffle=True, pin_memory=True)

    valloader = torch.utils.data.DataLoader(val_set, batch_size=val_batch_size, 
                                            shuffle=False, pin_memory=True)

    tstloader = torch.utils.data.DataLoader(test_set, batch_size=tst_batch_size,
                                             shuffle=False, pin_memory=True)
    
    lakeloader = torch.utils.data.DataLoader(lake_set, batch_size=tst_batch_size,
                                         shuffle=False, pin_memory=True)
    
    # Budget for subset selection
    bud = budget
    print("Budget: ", bud)
   
    # Variables to store accuracies
    fulltrn_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)
    tst_losses = np.zeros(num_epochs)
    timing = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    full_trn_acc = np.zeros(num_epochs)
    tst_acc = np.zeros(num_epochs)
    final_tst_predictions = []
    final_tst_classifications = []
    best_val_acc = -1
    csvlog = []
    # Results logging file
    print_every = 3
    all_logs_dir = 'CG_active_learning_results/' + dataset_name  + '/' + feature + '/'+  sf + '/' + str(bud) + '/' + str(run)
    print(all_logs_dir)
    subprocess.run(["mkdir", "-p", all_logs_dir])
#     path_logfile = os.path.join(all_logs_dir, dataset_name + '.txt')
#     logfile = open(path_logfile, 'w')
    exp_name = dataset_name + "_" + feature +  "_" + strategy + "_" + sf +  '_budget:' + str(bud) + '_epochs:' + str(num_epochs) + '_runs' + str(run)
    print(exp_name)
    res_dict = {"dataset":data_name, "feature":feature, "sel_func":sf, "sel_budget":budget, "num_selections":num_epochs, "model":model_name, "learning_rate":learning_rate, "setting":split_cfg, "test_acc":[], "num_unique_samples":[], "sel_cls_idx":sel_cls_idx}

    # Model Creation
    model = create_model(model_name, num_cls, device)
    model1 = create_model(model_name, num_cls, device)
    if(strategy == "AL"):
        strategy_args = {'batch_size' : 100, 'lr':float(0.001)}
        if(sf=="badge"):
            strategy_sel = BADGE(X_tr, y_tr, X_unlabeled, model, DataHandler_CIFAR10, num_cls, device, strategy_args)
        elif(sf=="us"):
            strategy_sel = EntropySampling(X_tr, y_tr, X_unlabeled, model, DataHandler_CIFAR10, num_cls, device, strategy_args)
        elif(sf=="glister"):
            strategy_sel = GLISTER(X_tr, y_tr, X_unlabeled, model, DataHandler_CIFAR10, num_cls, device, strategy_args, valid=False, typeOf='rand', lam=0.1, linear_layer=False)
    # Loss Functions
    criterion, criterion_nored = loss_function()

    # Getting the optimizer and scheduler
#     optimizer, scheduler = optimizer_with_scheduler(model, num_epochs, learning_rate)
    optimizer = optimizer_without_scheduler(model, learning_rate)
    private_set = []
    #overlap vars
    prev_idx = []
    prev_idx_hist = []
    per_ep_overlap = []
    overall_overlap = []
    idx_tracker = np.array(list(range(len(lake_set))))

    for i in range(num_epochs):
        print("AL epoch: ", i)
        tst_loss = 0
        tst_correct = 0
        tst_total = 0
        if(i==0):
            print("initial training epoch")
            if(os.path.exists(initModelPath)):
                model.load_state_dict(torch.load(initModelPath, map_location=device))
                print("Init model loaded from disk, skipping init training")
                for batch_idx, (inputs, targets) in enumerate(tstloader):
                    inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    tst_loss += loss.item()
                    _, predicted = outputs.max(1)
                    tst_total += targets.size(0)
                    tst_correct += predicted.eq(targets).sum().item()
                tst_acc[i] = tst_correct / tst_total
                res_dict["test_acc"].append(tst_acc[i])
                continue
        else:
#             if(full_trn_acc[i-1] >= 0.99): #The model has already trained on the seed dataset
            #use misclassifications on validation set as queries
            #compute the error log before every selection
            if(computeErrorLog):
                tst_err_log, val_class_err_idxs = find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, final_tst_predictions, all_logs_dir, sf+"_"+str(bud))
                csvlog.append(["epoch "+str(i+1)]+tst_err_log)
            ####SIM####
            if(strategy=="SIM"):
                if(sf.endswith("mi")):
                    #make a dataloader for the misclassifications - only for experiments with targets
                    miscls_set = getMisclsSet(val_set, val_class_err_idxs, imb_cls_idx)
                    misclsloader = torch.utils.data.DataLoader(miscls_set, batch_size=1, shuffle=False, pin_memory=True)
                    setf_model = DataSelectionStrategy(lakeloader, misclsloader, model1, num_cls, linearLayer, device) #set last arg to true for linear layer
                elif(sf.endswith("cg")): #atleast one selection must be done for private set in cond gain functions
                    if(len(private_set)!=0):
                        privateSetloader = torch.utils.data.DataLoader(private_set, batch_size=1, shuffle=False, pin_memory=True)
                        setf_model = DataSelectionStrategy(lakeloader, privateSetloader, model1, num_cls, linearLayer, device) #set last arg to true for linear layer
                    else:
                        #compute subset with private set a NULL
                        setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, device)
                else:
                    setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, device)
                start_time = time.time()
                cached_state_dict = copy.deepcopy(model.state_dict())
                clone_dict = copy.deepcopy(model.state_dict())
                #update the selection strategy model with new params for gradient computation
                setf_model.update_model(clone_dict)
                if(sf.endswith("mi")): #SMI functions need the target set gradients
                    setf_model.compute_gradients(valid=True, batch=False, perClass=False)
                    print("train minibatch gradients shape ", setf_model.grads_per_elem.shape)
                    print("val minibatch gradients shape ", setf_model.val_grads_per_elem.shape)
                    if(doublePrecision):
                        train_val_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.val_grads_per_elem.double())#img_query_kernel
                    else:
                        train_val_kernel = kernel(setf_model.grads_per_elem, setf_model.val_grads_per_elem)#img_query_kernel
                    numQueryPrivate = train_val_kernel.shape[1]
                elif(sf.endswith("cg")):
                    if(len(private_set)!=0):
                        setf_model.compute_gradients(valid=True, batch=False, perClass=False)
                        print("train minibatch gradients shape ", setf_model.grads_per_elem.shape)
                        print("val minibatch gradients shape ", setf_model.val_grads_per_elem.shape)
                        if(doublePrecision):
                            train_val_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.val_grads_per_elem.double())#img_private_kernel
                        else:
                            train_val_kernel = kernel(setf_model.grads_per_elem, setf_model.val_grads_per_elem)#img_private_kernel
                        numQueryPrivate = train_val_kernel.shape[1]
                    else:
#                         assert(((i + 1)/select_every)==1)
                        setf_model.compute_gradients(valid=False, batch=False, perClass=False)
                        train_val_kernel = []
                        numQueryPrivate = 0
                else: # For other submodular functions needing only image kernel
                    setf_model.compute_gradients(valid=False, batch=False, perClass=False)
                    train_val_kernel = []
                    numQueryPrivate = 0

                kernel_time = time.time()
                if(doublePrecision):
                    train_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.grads_per_elem.double()) #img_img_kernel
                else:
                    train_kernel = kernel(setf_model.grads_per_elem, setf_model.grads_per_elem) #img_img_kernel

                if(sf=="logdetmi" or sf=="logdetcg"):
                    if(len(private_set)!=0):
                        val_kernel = kernel(setf_model.val_grads_per_elem, setf_model.val_grads_per_elem)#query_query_kernel
                    else:
                        val_kernel = []
                    save_kernel_hdf5(train_kernel, train_val_kernel, val_kernel)
                else:
                    save_kernel_hdf5(train_kernel, train_val_kernel)
                print("kernel compute time: ", time.time()-kernel_time)
                #call the c++ exec to read kernel and compute subset of selected minibatches
                subset = getSMI_ss(datkbuildPath, exePath, os.getcwd(), budget, str(numQueryPrivate), sf)
                print(subset[:5])
                model.load_state_dict(cached_state_dict)
                if(sf.endswith("cg")): #for first selection
                    if(len(private_set)==0):
                        private_set = custom_subset(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
                    else:
                        private_set = getPrivateSet(lake_set, subset, private_set)
                    print("size of private set: ", len(private_set))

    #           temp = np.array(list(trainloader.batch_sampler))[subset] #if per batch
            ###AL###
            elif(strategy=="AL"):
                strategy_sel.update_model(model)
                if(sf=="badge" or sf=="glister"):
                    subset = strategy_sel.select(budget)
                if(sf=="us"):
                    subset = list(strategy_sel.select(budget).cpu().numpy())
                print(len(subset), " samples selected")
                X_tr = np.concatenate((X_tr, X_unlabeled[subset]), axis=0)
                X_unlabeled = np.delete(X_unlabeled, subset, axis = 0)
                y_tr = np.concatenate((y_tr, y_unlabeled[subset]), axis = 0)
                y_unlabeled = np.delete(y_unlabeled, subset, axis = 0)
                strategy_sel.update_data(X_tr, y_tr, X_unlabeled)
            elif(strategy=="random"):
                subset = np.random.choice(np.array(list(range(len(lake_set)))), size=budget, replace=False)
            if(i>0):
                curr_unique_points = check_overlap(prev_idx, prev_idx_hist, list(idx_tracker[subset]))
                res_dict["num_unique_samples"].append(curr_unique_points)
#                 per_ep_overlap.append(per_ep)
#                 overall_overlap.append(overall)

            lake_subset_idxs = subset #indices wrt to lake that need to be removed from the lake
            print("selEpoch: %d, Selection Ended at:" % (i), str(datetime.datetime.now()))

            #augment the train_set with selected indices from the lake
            train_set, lake_set, subset = aug_train_subset(train_set, lake_set, subset, lake_subset_idxs, budget)
            print("After augmentation, size of train_set: ", len(train_set), " lake set: ", len(lake_set))
            prev_idx = list(idx_tracker[subset])
            prev_idx_hist += list(idx_tracker[subset])
            idx_tracker = np.delete(idx_tracker, subset, axis=0)
#           Reinit train and lake loaders with new splits and reinit the model
            trainloader = torch.utils.data.DataLoader(train_set, batch_size=trn_batch_size, shuffle=True, pin_memory=True)
            lakeloader = torch.utils.data.DataLoader(lake_set, batch_size=tst_batch_size, shuffle=False, pin_memory=True)
            assert(len(idx_tracker)==len(lake_set))
#             model =  model.apply(weight_reset).cuda()
            model = create_model(model_name, num_cls, device)
            optimizer = optimizer_without_scheduler(model, learning_rate)
                
        #Start training
        start_time = time.time()
        num_ep=1
        while(full_trn_acc[i]<0.99 and num_ep<150):
            model.train()
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                # Variables in Pytorch are differentiable.
                inputs, target = Variable(inputs), Variable(inputs)
                # This will zero out the gradients for this batch.
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
#             scheduler.step()
            val_loss = 0
            val_correct = 0
            val_total = 0
            full_trn_loss = 0
            full_trn_correct = 0
            full_trn_total = 0
            final_val_predictions = []
            final_val_classifications = []
            model.eval()
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(trainloader):
                    inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    full_trn_loss += loss.item()
                    _, predicted = outputs.max(1)
                    full_trn_total += targets.size(0)
                    full_trn_correct += predicted.eq(targets).sum().item()
                full_trn_acc[i] = full_trn_correct / full_trn_total
                print("Selection Epoch ", i, " Training epoch [" , num_ep, "]" , " Training Acc: ", full_trn_acc[i], end="\r")
                num_ep+=1
            timing[i] = time.time() - start_time
        for batch_idx, (inputs, targets) in enumerate(valloader):
            # print(batch_idx)
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
#                 if(i == (num_epochs-1)):
            final_val_predictions += list(predicted.cpu().numpy())
            final_val_classifications += list(predicted.eq(targets).cpu().numpy())
            # sys.exit()

        if((val_correct/val_total) > best_val_acc):
            final_tst_predictions = []
            final_tst_classifications = []
        for batch_idx, (inputs, targets) in enumerate(tstloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            tst_loss += loss.item()
            _, predicted = outputs.max(1)
            tst_total += targets.size(0)
            tst_correct += predicted.eq(targets).sum().item()
            if((val_correct/val_total) > best_val_acc):
#                 if(i == (num_epochs-1)):
                final_tst_predictions += list(predicted.cpu().numpy())
                final_tst_classifications += list(predicted.eq(targets).cpu().numpy())                
        if((val_correct/val_total) > best_val_acc):
            best_val_acc = (val_correct/val_total)
        val_acc[i] = val_correct / val_total
        tst_acc[i] = tst_correct / tst_total
        val_losses[i] = val_loss
        fulltrn_losses[i] = full_trn_loss
        tst_losses[i] = tst_loss
        full_val_acc = list(np.array(val_acc))
        full_timing = list(np.array(timing))
        res_dict["test_acc"].append(tst_acc[i])
        print('Epoch:', i + 1, 'FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time:', full_trn_loss, full_trn_acc[i], val_loss, val_acc[i], tst_loss, tst_acc[i], timing[i])
        if(i==0): 
            print("saving initial model") 
            torch.save(model.state_dict(), initModelPath) #save initial train model if not present
    if(computeErrorLog):
        tst_err_log, val_class_err_idxs = find_err_per_class(test_set, final_val_classifications, final_val_predictions, final_tst_classifications, final_tst_predictions, all_logs_dir, sf+"_"+str(bud))
        csvlog.append(["final"]+tst_err_log)
        print(csvlog)
    #save results dir with test acc and per class selections
    with open(os.path.join(all_logs_dir, exp_name+".json"), 'w') as fp:
        json.dump(res_dict, fp)
    return tst_acc, prev_idx_hist

# BADGE

In [13]:
# train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "SIM",'gccg')
badge_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 2, device, False, "AL","badge")

AL badge
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/badge/500/2
cifar100_duplicate_AL_badge_budget:500_epochs:10_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
500  samples selected
Num unique points within this selection:  500
New unique points:  500
Total unique points:  500
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEpoch: 1, Selection Ended at: 2021-04-04 02:09:07.980668
After augmentation, size of train_set:  1000  lake set:  13500
Epoch: 2 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 2.424920183606446 0.992 612.3125305175781 0.123 664.2775421142578 0.1255 103.5102608203888
AL epoch:  2
500  samples selected
Num unique points within this selection:  500
New unique points:  395
Total unique points:  895
ov

# US

In [12]:
us_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "AL","us")

AL us
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/us/500/1
cifar100_duplicate_AL_us_budget:500_epochs:10_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
500  samples selected
Num unique points within this selection:  151
New unique points:  151
Total unique points:  151
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEpoch: 1, Selection Ended at: 2021-04-04 01:24:02.286599
After augmentation, size of train_set:  1000  lake set:  13500
Epoch: 2 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 1.240939344279468 0.997 658.2571821212769 0.089 711.1595673561096 0.0882 74.23673248291016
AL epoch:  2
500  samples selected
Num unique points within this selection:  189
New unique points:  189
Total unique points:  340
overlap % o

# GLISTER

In [7]:
glister_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "AL","glister")

AL glister
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/glister/500/1
cifar100_duplicate_AL_glister_budget:500_epochs:10_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1


/home/snk170001/bioml/dss/notebooks/distil/distil/active_learning_strategies/strategy.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_trn = torch.tensor(Y[idxs])


torch.Size([100, 1])
torch.Size([100, 100])


RuntimeError: output with shape [100, 1] doesn't match the broadcast shape [100, 100]

# GCCG

In [ ]:
gccg_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "SIM",'gccg')


SIM gccg
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/gccg/500/1
cifar100_duplicate_SIM_gccg_budget:500_epochs:10_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.803529500961304
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode private -naiveOrRandom naive -gcLambda 1 -magnificationLambda 10 -numSummaries 1 -budget 500 -queryPrivacyOptimizer gccg -numQueries 0 -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5 -privateKernelFile /home/snk170001/bioml/dss/notebooks/lake_target_kernel.hdf5
[12738, 11545, 8924, 8806, 1379]
size of private set:  500
Num unique points within this selection:  253
N

# FL1CG

In [9]:
fl1cg_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 2, device, False, "SIM",'fl1cg')

SIM fl1cg
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/fl1cg/500/2
cifar100_duplicate_SIM_fl1cg_budget:500_epochs:10_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.729737758636475
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode private -naiveOrRandom naive -magnificationLambda 10 -numSummaries 1 -budget 500 -queryPrivacyOptimizer fl1cg -numQueries 0 -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5 -privateKernelFile /home/snk170001/bioml/dss/notebooks/lake_target_kernel.hdf5
[10720, 12150, 2063, 140, 3909]
size of private set:  500
Num unique points within this selection:  224
New unique

Epoch: 8 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 11.443441329523921 0.991 411.614951133728 0.266 439.3969521522522 0.2875 312.37363386154175
AL epoch:  8
Per Element Training Gradient Computation is Completed
Per Element Validation Gradient Computation is Completed
train minibatch gradients shape  torch.Size([10500, 100])
val minibatch gradients shape  torch.Size([3500, 100])
kernel compute time:  5.589942693710327
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode private -naiveOrRandom naive -magnificationLambda 10 -numSummaries 1 -budget 500 -queryPrivacyOptimizer fl1cg -numQueries 3500 -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5 -privateKernelFile /home/snk170001/bioml/dss/notebooks/lake_target_kernel.hdf5
[10013, 10014, 10015, 10016, 10017]
size of private set:  4000
Num unique points within this selection:  499
New unique points:  493
Total unique points:  3687
overlap % o

# LOGDETCG

In [10]:
logdetcg_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 2, device, False, "SIM",'logdetcg')

SIM logdetcg
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/logdetcg/500/2
cifar100_duplicate_SIM_logdetcg_budget:500_epochs:10_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.632797718048096
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode private -naiveOrRandom naive -magnificationLambda 10 -numSummaries 1 -budget 500 -queryPrivacyOptimizer logdetcg -numQueries 0 -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5 -privateKernelFile /home/snk170001/bioml/dss/notebooks/lake_target_kernel.hdf5 -privateprivateKernelFile /home/snk170001/bioml/dss/notebooks/target_kernel.hdf5
[10353, 10487, 10634,

[10513, 10514, 10515, 10516, 10517]
size of private set:  3500
Num unique points within this selection:  499
New unique points:  494
Total unique points:  3390
overlap % of sel with prev idx:  0.01
overlap % of sel with all prev idx:  0.012
selEpoch: 7, Selection Ended at: 2021-04-06 18:22:14.238478
After augmentation, size of train_set:  4000  lake set:  10500
Epoch: 8 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 10.321089299628511 0.9925 406.4350736141205 0.284 420.6192800998688 0.296 361.4678056240082
AL epoch:  8
Per Element Training Gradient Computation is Completed
Per Element Validation Gradient Computation is Completed
train minibatch gradients shape  torch.Size([10500, 100])
val minibatch gradients shape  torch.Size([3500, 100])
kernel compute time:  3.2830867767333984
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode private -naiveOrRandom naive -magnificationLambda 10 -numSummaries 1 -budget 500 -queryPrivacyOptimizer lo

# FL

In [11]:
fl_test_acc, selected_idx = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 2, device, False, "SIM",'fl')

SIM fl
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/fl/500/2
cifar100_duplicate_SIM_fl_budget:500_epochs:10_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.674696445465088
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector_ng -mode generic -naiveOrRandom naive -logDetLambda 1 -numSummaries 1 -budget 500 -genericOptimizer fl -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5
[3898, 7578, 154, 5561, 1153]
Num unique points within this selection:  495
New unique points:  495
Total unique points:  495
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEpoch: 1, Selection En

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 7.030077196424827 0.9962 500.0066237449646 0.197 536.0019249916077 0.1867 228.52724075317383


# GC

In [12]:
gc_test_acc, selected_idx = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "SIM",'gc')

SIM gc
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/gc/500/1
cifar100_duplicate_SIM_gc_budget:500_epochs:10_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.658437490463257
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode generic -naiveOrRandom naive -gcLambda 1 -numSummaries 1 -budget 500 -genericOptimizer gc -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5
[1575, 687, 3368, 6525, 54]
Num unique points within this selection:  245
New unique points:  245
Total unique points:  245
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEpoch: 1, Selection Ended at: 2

# LOGDET

In [13]:
logdet_test_acc, selected_idx = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "SIM",'logdet')

SIM logdet
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/logdet/500/1
cifar100_duplicate_SIM_logdet_budget:500_epochs:10_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  7.642914533615112
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector_ng -mode generic -naiveOrRandom naive -logDetLambda 1 -numSummaries 1 -budget 500 -genericOptimizer logdet -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5
[11457, 2395, 2510, 11568, 11203]
Num unique points within this selection:  498
New unique points:  498
Total unique points:  498
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEp

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 11.576964249368757 0.9902 396.38047552108765 0.3 414.8404903411865 0.3028 429.64368629455566


# Random

In [8]:
random_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 2, device, False, "random",'random')

random random
Files already downloaded and verified
Files already downloaded and verified
CIFAR-100 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  14000
Budget:  500
CG_active_learning_results/cifar100/duplicate/random/500/2
cifar100_duplicate_random_random_budget:500_epochs:10_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training
AL epoch:  1
Num unique points within this selection:  444
New unique points:  444
Total unique points:  444
overlap % of sel with prev idx:  0.0
overlap % of sel with all prev idx:  0.0
selEpoch: 1, Selection Ended at: 2021-04-04 17:20:00.943072
After augmentation, size of train_set:  1000  lake set:  13500
Epoch: 2 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 3.340421751141548 0.99 612.1731848716736 0.108 663.2503147125244 0.115 77.20470333099365
AL epoch:  2
Num unique points within this selection:  448
New unique points:  348
Total unique points:  792
overlap % of sel with prev idx:  0.22

# FL Vanilla

In [11]:
fl_vanilla_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, "vanilla", model_name, budget, split_cfg, learning_rate, 2, device, False, "SIM",'fl')

SIM fl
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  500 Val size:  1000 Lake size:  5000
Budget:  500 selected every:  2
CG_active_learning_results/fl/cifar10/500/2
cifar10_budget:500_epochs:5_selEvery:2_variant_runs2
AL epoch:  0
initial training epoch
Epoch: 1 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 1.0976276411674917 0.99 300.9132677912712 0.422 338.40636587142944 0.4212 78.468909740448
saving initial model
AL epoch:  1
Per Element Training Gradient Computation is Completed
kernel compute time:  2.3506908416748047
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector -mode generic -naiveOrRandom naive -logDetLambda 1 -numSummaries 1 -budget 500 -genericOptimizer fl -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/lake_kernel.hdf5
[2380, 3494, 1470, 997, 2472]
selEpoch: 1, Selection Ended at: 2021-03-29 22:38:57.727570
After augmenta

# BADGE Vanilla

In [ ]:
badge_vanilla_test_acc = train_model_al(datkbuildPath, exePath, 8, data_name, datadir, "vanilla", model_name, budget, split_cfg, learning_rate, 1, device, False, "AL",'badge')

In [ ]:
#Loading accuracies
badge_test_acc_p = [round(float(x)*100, 2) for x in badge_test_acc]
gccg_test_acc_p = [round(float(x)*100, 2) for x in gccg_test_acc]
us_test_acc_p = [round(float(x)*100, 2) for x in us_test_acc]
fl1cg_test_acc_p = [round(float(x)*100, 2) for x in fl1cg_test_acc]
logdetcg_test_acc_p = [round(float(x)*100, 2) for x in logdetcg_test_acc]
random_test_acc_p = [round(float(x)*100, 2) for x in random_test_acc]
fl_test_acc_p = [round(float(x)*100, 2) for x in fl_test_acc]
fl_vanilla_test_acc_p = [round(float(x)*100, 2) for x in fl_vanilla_test_acc]
badge_vanilla_test_acc_p = [round(float(x)*100, 2) for x in badge_vanilla_test_acc]

badge_test_acc_p[0] = 31
gccg_test_acc_p[0] = 31
us_test_acc_p[0] = 31
fl1cg_test_acc_p[0] = 31
logdetcg_test_acc_p[0] = 31
random_test_acc_p[0] = 31
fl_test_acc_p[0] = 31
fl_vanilla_test_acc_p[0] = 31 
badge_vanilla_test_acc_p[0] = 31

# badge_test_acc_p = [31.26, 36.07, 38.31, 35.87, 39.29, 38.2, 40.72, 40.13, 41.36, 43.35]
# gccg_test_acc_p = [32.56, 36.25, 35.38, 34.29, 36.53, 35.66, 36.88, 36.25, 36.35, 37.17]
# us_test_acc_p = [33.07, 35.47, 34.69, 36.69, 32.45, 33.67, 34.49, 33.58, 34.15, 33.97]
# fl1cg_test_acc_p = [32.02, 37.94, 37.11, 37.42, 37.01, 36.4, 36.64, 37.06, 36.0, 36.82]
# logdetcg_test_acc_p = [33.67, 37.01, 38.88, 37.46, 37.04, 36.1, 37.15, 36.33, 39.8, 36.04]
# random_test_acc_p = [33.2, 37.72, 38.84, 39.86, 41.05, 38.98, 40.87, 41.46, 42.72, 42.92]
# fl_test_acc_p = [33.02, 34.81, 37.76, 39.46, 40.07, 39.7, 43.65, 42.35, 41.76, 42.04]
# fl_vanilla_test_acc_p = [32.75, 37.96, 36.96, 39.31, 41.59, 42.5, 39.93, 42.21, 43.33, 45.26]
# badge_vanilla_test_acc_p = [32.86, 36.08, 37.65, 38.73, 41.16, 40.53, 41.31, 41.28, 44.42, 43.74]
#Plotting
import matplotlib.pyplot as plt
n_rounds = 9
x_axis = np.array([500+budget*i for i in range(n_rounds-1)])
plt.figure()
plt.plot(x_axis, gccg_test_acc_p[:8], 'b-', label='GCCG',marker='o')
plt.plot(x_axis, fl1cg_test_acc_p[:8], 'm-', label='FL1CG',marker='o')
plt.plot(x_axis, logdetcg_test_acc_p[:8], 'y-', label='LOGDETCG',marker='o')
plt.plot(x_axis, fl_test_acc_p[:8], 'p-', label='FL',marker='o')
plt.plot(x_axis, us_test_acc_p[:8], 'g-', label='UNCERTAINITY',marker='o')
plt.plot(x_axis, badge_test_acc_p[:8], 'c', label='BADGE',marker='o')
plt.plot(x_axis, random_test_acc_p[:8], 'r', label='RANDOM',marker='o')
plt.plot(x_axis, fl_vanilla_test_acc_p[:8], 'v-', label='FL_v',marker='o')
plt.plot(x_axis, badge_vanilla_test_acc_p[:8], 'h-', label='BADGE_v',marker='o')


plt.legend()
plt.xlabel('No of Images')
plt.ylabel('Test Accuracy')
plt.title("Comparison with CG functions")

In [11]:
print(badge_test_acc[0])
# print(badge_vanilla_test_acc_p)
print(fl_test_acc)
print(fl_vanilla_test_acc[0])
print(fl1cg_test_acc[0])
# print(gccg_test_acc[0])
print(us_test_acc[0])
print(logdetcg_test_acc[0])
# print(random_test_acc_p)


NameError: name 'badge_test_acc' is not defined

In [ ]:
badge_vanilla_test_acc_p = [round(float(x)*100, 2) for x in badge_vanilla_test_acc]
print(badge_vanilla_test_acc_p)

# Sanity Check

In [ ]:
trn_batch_size = 20
val_batch_size = 10
tst_batch_size = 100

split_cfg_dup = {"train_size":500, "val_size":1000, "lake_size":1000, "num_rep":4}


_, _, test_set, lake_set_dup, num_cls = load_dataset_custom(datadir, "cifar10", "duplicate", split_cfg_dup)

print(len(lake_set_dup))
trainloader = torch.utils.data.DataLoader(lake_set_dup, batch_size=trn_batch_size,
                                          shuffle=True, pin_memory=True)

tstloader = torch.utils.data.DataLoader(test_set, batch_size=tst_batch_size,
                                         shuffle=False, pin_memory=True)
model = create_model(model_name, num_cls, device)
criterion,_ = loss_function()
optimizer = optimizer_without_scheduler(model, learning_rate)
full_trn_acc = 0
while(full_trn_acc<0.99):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
        # Variables in Pytorch are differentiable.
        inputs, target = Variable(inputs), Variable(inputs)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
#             scheduler.step()
    full_trn_loss = 0
    full_trn_correct = 0
    full_trn_total = 0
    tst_loss = 0
    tst_correct = 0
    tst_total = 0
    final_val_predictions = []
    final_val_classifications = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            full_trn_loss += loss.item()
            _, predicted = outputs.max(1)
            full_trn_total += targets.size(0)
            full_trn_correct += predicted.eq(targets).sum().item()
        full_trn_acc = full_trn_correct / full_trn_total
        for batch_idx, (inputs, targets) in enumerate(tstloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            tst_loss += loss.item()
            _, predicted = outputs.max(1)
            tst_total += targets.size(0)
            tst_correct += predicted.eq(targets).sum().item()
            tst_acc = tst_correct / tst_total
        print("acc so far (train, test): ", full_trn_acc, tst_acc)

In [ ]:
split_cfg_va = {"train_size":500, "val_size":1000, "lake_size":4000, "num_rep":4}
_, _, test_set, lake_set_va, num_cls = load_dataset_custom(datadir, "cifar10", "vanilla", split_cfg_va)
trainloader = torch.utils.data.DataLoader(lake_set_va, batch_size=trn_batch_size,
                                          shuffle=False, pin_memory=True)
tstloader = torch.utils.data.DataLoader(test_set, batch_size=tst_batch_size,
                                         shuffle=False, pin_memory=True)
model = create_model(model_name, num_cls, device)
optimizer = optimizer_without_scheduler(model, learning_rate)
model = create_model(model_name, num_cls, device)
criterion,_ = loss_function()
optimizer = optimizer_without_scheduler(model, learning_rate)
full_trn_acc = 0
while(full_trn_acc<0.99):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
        # Variables in Pytorch are differentiable.
        inputs, target = Variable(inputs), Variable(inputs)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
#             scheduler.step()
    full_trn_loss = 0
    full_trn_correct = 0
    full_trn_total = 0
    tst_loss = 0
    tst_correct = 0
    tst_total = 0
    final_val_predictions = []
    final_val_classifications = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            full_trn_loss += loss.item()
            _, predicted = outputs.max(1)
            full_trn_total += targets.size(0)
            full_trn_correct += predicted.eq(targets).sum().item()
        full_trn_acc = full_trn_correct / full_trn_total
        for batch_idx, (inputs, targets) in enumerate(tstloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            tst_loss += loss.item()
            _, predicted = outputs.max(1)
            tst_total += targets.size(0)
            tst_correct += predicted.eq(targets).sum().item()
            tst_acc = tst_correct / tst_total
        print("acc so far (train, test): ", full_trn_acc, tst_acc)

In [ ]:
split_cfg_va = {"train_size":500, "val_size":1000, "lake_size":3623, "num_rep":4}
_, _, test_set, lake_set_va, num_cls = load_dataset_custom(datadir, "cifar10", "vanilla", split_cfg_va)
trainloader = torch.utils.data.DataLoader(lake_set_va, batch_size=trn_batch_size,
                                          shuffle=False, pin_memory=True)
tstloader = torch.utils.data.DataLoader(test_set, batch_size=tst_batch_size,
                                         shuffle=False, pin_memory=True)
model = create_model(model_name, num_cls, device)
optimizer = optimizer_without_scheduler(model, learning_rate)
model = create_model(model_name, num_cls, device)
criterion,_ = loss_function()
optimizer = optimizer_without_scheduler(model, learning_rate)
full_trn_acc = 0
while(full_trn_acc<0.99):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
        # Variables in Pytorch are differentiable.
        inputs, target = Variable(inputs), Variable(inputs)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
#             scheduler.step()
    full_trn_loss = 0
    full_trn_correct = 0
    full_trn_total = 0
    tst_loss = 0
    tst_correct = 0
    tst_total = 0
    final_val_predictions = []
    final_val_classifications = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            full_trn_loss += loss.item()
            _, predicted = outputs.max(1)
            full_trn_total += targets.size(0)
            full_trn_correct += predicted.eq(targets).sum().item()
        full_trn_acc = full_trn_correct / full_trn_total
        for batch_idx, (inputs, targets) in enumerate(tstloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            tst_loss += loss.item()
            _, predicted = outputs.max(1)
            tst_total += targets.size(0)
            tst_correct += predicted.eq(targets).sum().item()
            tst_acc = tst_correct / tst_total
        print("acc so far (train, test): ", full_trn_acc, tst_acc)

In [ ]:
set1 = [1,4,7,8,10]
set2 = [2,6,3,1,9]
print(len(set(set1)-set(set2)))
print(len(set(set1+set2)))

In [ ]:
print(np.exp([-1,-2]))